In [1]:
!pip install -q langchain langchain_core langchain_community sentence_transformers faiss-cpu langchain_groq langchain_huggingface pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
groq_api_key = 'GROQ_API_KEY'
model = "gemma2-9b-it"

In [5]:
pdf_path = '/content/Knowledge Base.pdf'

In [6]:
loader = PyPDFLoader(pdf_path)
document = loader.load()

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(document)

In [8]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [10]:
groq_chat = ChatGroq(
  groq_api_key=groq_api_key,
  model_name=model
)

In [33]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, output_key='answer'
)

In [34]:
chain = ConversationalRetrievalChain.from_llm(
    llm = groq_chat,
    chain_type = 'stuff',
    retriever = vectorstore.as_retriever(),
    memory = memory,
    return_source_documents=True,
    output_key="answer"
)

In [38]:
user_input = input("Ask your Question: ")

Ask your Question: It's prevention


In [39]:
response = chain.invoke({"question": user_input, "chat_history": memory.load_memory_variables({})})
print(response["answer"])

To prevent citrus canker, you can: 

* Obtain plants from a nursery free from disease; avoid plants from low-altitude areas. 
* Prune and remove infected parts, and apply pesticide to the remaining healthy sections.
* During December–January, spray a 1% Bordeaux mixture or Copper Oxychloride at 2 -3 grams per liter of water. 



